## Plate Recognizer API:

Link for API Documentation - https://guides.platerecognizer.com/docs/snapshot/api-reference#snapshot-cloud-api

Get API Token from here - https://app.platerecognizer.com/service/snapshot-cloud/

**Defining imports:**

In [40]:
import requests
from pprint import pprint
from IPython.display import JSON
import json
import cv2
import os
import random
import pandas as pd
import time

**Calling API:**

In [9]:
def recognize_license_plate(image_path, token, regions=["sg"], strict_region = True, mmc=True):
    url = "https://api.platerecognizer.com/v1/plate-reader/"
    headers = {
        "Authorization": f"Token {token}"
    }
    
    # Open the image file
    with open(image_path, "rb") as fp:
        
        # Set up the data payload
        data = {
            "regions": regions,       # Specify relevant regions
            "mmc": str(mmc).lower()   # Enable make, model, color detection if set to True
        }
        if strict_region:
            data["config"] = json.dumps({"region": "strict"})  # Enforce strict region matching if needed

        # Send request with file upload
        response = requests.post(url, headers=headers, files={"upload": fp}, data=data)
        
        # Check response status and print results
        if response.status_code == 200 or response.status_code == 201:
            pprint(response.json())
            return response.json()
        else:
            print("Error:", response.status_code, response.text)
            return None

**Defining token for platerecognizer:**

In [10]:
token = "210ed0449ee06e8d9bcee4a67c742814e4e7366e"

**Selecting entire directory of images:**

In [14]:
current_directory = "selected_images"

# Get list of all image files in the folder
image_files = [f for f in os.listdir(current_directory) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [21]:
print(image_files)

['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '4.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']


In [27]:
# Results container
results = []
full_response = []

for image in image_files:
    image_path = os.path.join(current_directory, image)
    print(f"Processing: {image}")
    response = recognize_license_plate(image_path, token)

    # Store the complete response
    full_response.append({
        "Image Number (Filename)": image,
        "Full Response": response
    })

    plate = make = model = None

    if response and response.get("results"):
        result = response["results"][0]
        plate = result.get("plate", None)

        if "model_make" in result and result["model_make"]:
            make = result["model_make"][0].get("make", None)
            model = result["model_make"][0].get("model", None)

    results.append({
        "Image Number (Filename)": image,
        "License Plate": plate.upper() if plate else None,
        "Make": make.title() if make else None,
        "Model": model.title() if model else None
    })

Processing: 1.jpg
{'camera_id': None,
 'filename': '1942_WmckM_1.jpg',
 'image_height': 1390,
 'image_width': 1854,
 'processing_time': 83.529,
 'results': [{'box': {'xmax': 1237, 'xmin': 701, 'ymax': 1026, 'ymin': 777},
              'candidates': [{'plate': 'sjm764s', 'score': 0.999}],
              'color': [{'color': 'silver', 'score': 0.792},
                        {'color': 'blue', 'score': 0.116},
                        {'color': 'white', 'score': 0.027}],
              'direction': 219,
              'direction_score': 0.697,
              'dscore': 0.855,
              'model_make': [{'make': 'Toyota',
                              'model': 'Corolla',
                              'score': 0.171},
                             {'make': 'generic',
                              'model': 'Unknown',
                              'score': 0.067}],
              'orientation': [{'orientation': 'Front', 'score': 0.857},
                              {'orientation': 'Rear', 'score': 

In [23]:
# Save full responses to a separate JSON file
with open("full_responses.json", "w") as f:
    json.dump(full_response, f, indent=2)

In [24]:
# Create DataFrame from full responses
full_response_df = pd.DataFrame(full_response)

# Save full responses to Excel
full_response_df.to_excel("full_responses.xlsx", index=False)

print("\n✅ Full responses saved to full_responses.json and full_responses.xlsx")


✅ Full responses saved to full_responses.json and full_responses.xlsx


In [28]:
print(results)

[{'Image Number (Filename)': '1.jpg', 'License Plate': 'SJM764S', 'Make': 'Toyota', 'Model': 'Corolla'}, {'Image Number (Filename)': '10.jpg', 'License Plate': 'SLU7648H', 'Make': 'Honda', 'Model': 'Odyssey'}, {'Image Number (Filename)': '11.jpg', 'License Plate': 'SKW5352A', 'Make': 'Honda', 'Model': 'Civic'}, {'Image Number (Filename)': '12.jpg', 'License Plate': 'GZB542E', 'Make': 'Toyota', 'Model': 'Hilux'}, {'Image Number (Filename)': '13.jpg', 'License Plate': 'SFJ1628J', 'Make': 'Mitsubishi', 'Model': 'Space Star'}, {'Image Number (Filename)': '14.jpg', 'License Plate': 'SLB103J', 'Make': 'Toyota', 'Model': 'Alphard'}, {'Image Number (Filename)': '15.jpg', 'License Plate': 'SJQ9144B', 'Make': 'Honda', 'Model': 'Stream'}, {'Image Number (Filename)': '16.jpg', 'License Plate': 'SLF7306Z', 'Make': 'Honda', 'Model': 'Hr-V'}, {'Image Number (Filename)': '17.jpg', 'License Plate': 'SMD1514C', 'Make': 'Nissan', 'Model': 'Rogue Sport'}, {'Image Number (Filename)': '18.jpg', 'License Pla

In [29]:
# Create DataFrame
df = pd.DataFrame(results)

# Save to Excel
df.to_excel("recognized_plates.xlsx", index=False)

# Save to JSON
df.to_json("recognized_plates.json", orient="records", indent=2)

print("Saved results to recognized_plates.xlsx and recognized_plates.json")

Saved results to recognized_plates.xlsx and recognized_plates.json


**Selecting x number of images:**

In [30]:
import os

# List all files and folders in the current directory
files = os.listdir('selected_images')
print(files)

['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '4.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', 'Truth_label.xlsx', '~$Truth_label.xlsx']


In [31]:
# Function to process a single image
def process_image(image_path, image_name, max_retries=2):
    for attempt in range(max_retries):
        print(f"\nProcessing: {image_name} (Attempt {attempt + 1})")
        response = recognize_license_plate(image_path, token)
        
        if response and response.get("results"):
            result = response["results"][0]
            plate = result.get("plate", None)
            
            if "model_make" in result and result["model_make"]:
                make = result["model_make"][0].get("make", None)
                model = result["model_make"][0].get("model", None)
            else:
                make = model = None
                
            if plate:  # If we got a valid plate, return the results
                return {
                    "Image Number (Filename)": image_name,
                    "License Plate": plate.upper(),
                    "Make": make.title() if make else None,
                    "Model": model.title() if model else None
                }, response
                
        if attempt < max_retries - 1:
            print(f"Retrying {image_name}...")
            time.sleep(1)  
            
  
    return {
        "Image Number (Filename)": image_name,
        "License Plate": None,
        "Make": None,
        "Model": None
    }, None


In [32]:
# Set the path to the current directory
current_directory = "selected_images"

# Get a list of all image files (jpg, png, etc.) in the current directory
image_files = [f for f in os.listdir(current_directory) if f.endswith(('.jpg', '.jpeg', '.png'))]

# --- Collect results ---
results = []
full_response = []

# Choose x random images from the list
random_images = random.sample(image_files, 2)

# Call the function for each selected image
for image in random_images:
    image_path = os.path.join(current_directory, image)
    result, response = process_image(image_path, image)
    
    results.append(result)
    
    if response:
        full_response.append({
            "Image Number (Filename)": image,
            "Full Response": response
        })


Processing: 21.jpg (Attempt 1)
{'camera_id': None,
 'filename': '1236_sLQfj_21.jpg',
 'image_height': 1536,
 'image_width': 2048,
 'processing_time': 120.049,
 'results': [{'box': {'xmax': 1246, 'xmin': 844, 'ymax': 848, 'ymin': 756},
              'candidates': [{'plate': 'slb1123u', 'score': 0.997}],
              'color': [{'color': 'silver', 'score': 0.915},
                        {'color': 'black', 'score': 0.02},
                        {'color': 'yellow', 'score': 0.012}],
              'direction': 70,
              'direction_score': 1.0,
              'dscore': 0.884,
              'model_make': [{'make': 'Nissan',
                              'model': 'Versa',
                              'score': 0.754},
                             {'make': 'generic',
                              'model': 'Unknown',
                              'score': 0.022}],
              'orientation': [{'orientation': 'Rear', 'score': 0.926},
                              {'orientation': 'Front

In [33]:
# Sort results by Image Number
results = sorted(results, key=lambda x: int(x["Image Number (Filename)"].split('.')[0]))
full_response = sorted(full_response, key=lambda x: int(x["Image Number (Filename)"].split('.')[0]))

In [34]:
print(results)

[{'Image Number (Filename)': '11.jpg', 'License Plate': 'SKW5352A', 'Make': 'Honda', 'Model': 'Civic'}, {'Image Number (Filename)': '21.jpg', 'License Plate': 'SLB1123U', 'Make': 'Nissan', 'Model': 'Versa'}]


In [37]:
# Save full responses to a separate JSON file
with open("full_responses.json", "w") as f:
    json.dump(full_response, f, indent=2)

In [38]:
# --- Convert results to DataFrame ---
df = pd.DataFrame(results)

In [39]:
# --- Save to Excel and JSON ---
df.to_excel("recognized_plates.xlsx", index=False)
df.to_json("recognized_plates.json", orient="records", indent=2)

print("\n✅ Results saved to recognized_plates.xlsx and recognized_plates.json")


✅ Results saved to recognized_plates.xlsx and recognized_plates.json
